In [11]:
import pandas as pd
import numpy as np
from django.db.models import F, FloatField, Sum

In [3]:
# Define data types for initial parsing of CSV
dtype = {
    'id': np.int64,
    'listing_id': np.int64,
    'date': str,
    'reviewer_id': np.int64,
    'comments': str
}
     
# Specify columns that should be parsed as dates
parse_dates = ['date']

# Specify conversion functions for particular columns
converters = {
}

In [4]:
# Read CSV contents into dataframe
df = pd.read_csv(
    '../res/reviews-3-aug-16.csv.gz', 
    usecols=list(dtype.keys()),
    dtype=dtype, 
    parse_dates=parse_dates, 
    converters=converters
)

In [5]:
# Check for columns containing null values
df.isnull().any()

listing_id     False
id             False
date           False
reviewer_id    False
comments        True
dtype: bool

In [5]:
# # Fill certain null / missing values
# df['reviews_per_month'].fillna(value=0.0, inplace=True)
# df['bathrooms'].fillna(value=0.0, inplace=True)
# df['bedrooms'].fillna(value=0.0, inplace=True)

# # Drop entries with certain missing values
# df.dropna(subset=['id', 'description', 'host_since', 'host_name'], inplace=True)

In [8]:
# Delete all existing Reviews
Review.objects.all().delete()

# Instantiate and save a Listing for each row in the dataframe
for index, row in df.iterrows():
    try:
        listing = Listing.objects.get(pk=row['listing_id'])
        Review(
            listing=listing,
            id=row['id'],
            date=row['date'],
            reviewer_id=row['reviewer_id'],
            comments=row['comments']
        ).save() # Commit to DB
    except Listing.DoesNotExist:
        print ("Listing %d does not exist" % row['listing_id'])
    if index % 1000 == 0:
        print('.', end='')

..........................................................................................................................................................................................................................................................Listing 2898119 does not exist
Listing 2898119 does not exist
.................Listing 10034835 does not exist
Listing 10034835 does not exist
...........................................................................................................................................Listing 6505664 does not exist
........................Listing 9998970 does not exist
..........

In [11]:
Review.objects.filter(comments="nan").update(comments="")

206

In [14]:
from django.db.models.functions import ExtractMonth
Review.objects.annotate(month=ExtractMonth('date')).filter(month=6).count()

53344

In [38]:
months=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct','Nov','Dec']
def print_monthly_review_totals(queryset):
    for i in range(1,13):
        print(months[i-1], "%.2f%%" % float(100 *
            queryset.annotate(month=ExtractMonth('date')).filter(month=i).count() / queryset.count()))

In [42]:
print("Booking activity for all listings")
print_monthly_review_totals(Review.objects.all())

Booking activity for all listings
Jan 6.77%
Feb 7.31%
Mar 8.77%
Apr 9.74%
May 11.16%
Jun 12.14%
Jul 13.94%
Aug 7.48%
Sep 5.94%
Oct 6.23%
Nov 5.25%
Dec 5.26%


In [41]:
print("Booking activity for Van Nuys")
listing_ids = [_['id'] for _ in Neighborhood.objects.get(name="Van Nuys").listing_set.all().values('id')]
print_monthly_review_totals(Review.objects.filter(listing_id__in=listing_ids))

Booking activity for Van Nuys
Jan 5.98%
Feb 5.23%
Mar 7.85%
Apr 10.65%
May 10.84%
Jun 14.86%
Jul 18.04%
Aug 5.61%
Sep 5.89%
Oct 4.58%
Nov 5.33%
Dec 5.14%
